# Bake-off: Word-level entailment with neural networks

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2018"

## Contents

0. [Overview](#Overview)
0. [Set-up](#Set-up)
0. [Data](#Data)
  0. [Edge disjoint](#Edge-disjoint)
  0. [Word disjoint](#Word-disjoint)
  0. [Word disjoint and balanced](#Word-disjoint-and-balanced)
0. [Baseline](#Baseline)
  0. [Representing words: vector_func](#Representing-words:-vector_func)
  0. [Combining words into inputs: vector_combo_func](#Combining-words-into-inputs:-vector_combo_func)
  0. [Classifier model](#Classifier-model)
  0. [Baseline results](#Baseline-results)
0. [Bake-off submission](#Bake-off-submission)

## Overview

__Problem__: Word-level natural language inference.

Training examples are pairs of words $(w_{L}, w_{R}), y$ with $y$ a relation in

* __synonym__: very roughly identical meanings; symmetric
* __hyponym__: e.g., _puppy_ is a hyponym of _dog_
* __hypernym__:  e.g., _dog_ is a hypernym of _puppy_
* __antonym__: semantically opposed within a domain; symmetric

The dataset is due to [Bowman et al. 2015](https://arxiv.org/abs/1406.1827). See [below](#Data) for details on how it was processed for this bake-off.

## Set-up

0. Make sure your environment includes all the requirements for [the cs224u repository](https://github.com/cgpotts/cs224u/).

0. Make sure you have the [the Wikipedia 2014 + Gigaword 5 distribution](http://nlp.stanford.edu/data/glove.6B.zip) of  pretrained GloVe vectors downloaded and unzipped, and that `glove_home` below is pointing to it.

0. Make sure `wordentail_filename` below is pointing to the full path for `nli_wordentail_bakeoff_data.json`, which is included in [the nlidata.zip archive](http://web.stanford.edu/class/cs224u/data/nlidata.zip).

In [1]:
from collections import defaultdict
import json
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tf_shallow_neural_classifier import TfShallowNeuralClassifier
import nli
import utils

In [2]:
nlidata_home = 'nlidata'

wordentail_filename = os.path.join(
    nlidata_home, 'nli_wordentail_bakeoff_data.json')

glove_home = os.path.join("vsmdata", "glove.6B")

## Data

As noted above, the dataset was originally released by [Bowman et al. 2015](https://arxiv.org/abs/1406.1827), who derived it from [WordNet](https://wordnet.princeton.edu) using some heuristics (and thus it might contain some errors or unintuitive pairings).

I've processed the data into three different train/test splits, in an effort to put some pressure on our models to actually learn these semantic relations, as opposed to exploiting regularities in the sample.

* `edge_disjoint`: The `train` and `dev` __edge__ sets are disjoint, but many __words__ appear in both `train` and `dev`.
* `word_disjoint`: The `train` and `dev` __vocabularies are disjoint__, and thus the edges are disjoint as well.
* `word_disjoint_balanced`: Like `word_disjoint`, but with each word appearing at most one time as the left word and at most one time on the right for a given relation type.

These are progressively harder problems

* For `word_disjoint`, there is real pressure on the model to learn abstract relationships, as opposed to memorizing properties of individual words.

* For `word_disjoint_balanced`, the model can't even learn that some terms tend to appear more on the left or the right. This might be a step too far. For example, appearing more on the right for `hypernym` corresponds in a deep way with being a more general term, which is a non-trivial lexical property that we want our models to learn.

In [66]:
with open(wordentail_filename) as f:
    wordentail_data = json.load(f)

The outer keys are the three splits plus a list giving the vocabulary for the entire dataset:

In [4]:
wordentail_data.keys()

dict_keys(['vocab', 'word_disjoint_balanced', 'word_disjoint', 'edge_disjoint'])

### Edge disjoint

In [5]:
wordentail_data['edge_disjoint'].keys()

dict_keys(['dev', 'train'])

This is what the split looks like; all three have this same format:

In [6]:
wordentail_data['edge_disjoint']['dev'][: 5]

[[['archived', 'records'], 'synonym'],
 [['stage', 'station'], 'synonym'],
 [['engineers', 'design'], 'hypernym'],
 [['save', 'book'], 'hypernym'],
 [['match', 'supply'], 'hypernym']]

Let's test to make sure no edges are shared between `train` and `dev`:

In [7]:
nli.get_edge_overlap_size(wordentail_data, 'edge_disjoint')

0

As we expect, a *lot* of vocabulary items are shared between `train` and `dev`:

In [8]:
nli.get_vocab_overlap_size(wordentail_data, 'edge_disjoint')

4769

This is a large percentage of the entire vocab:

In [9]:
len(wordentail_data['vocab'])

6560

Here's the distribution of labels in the `train` set. It's highly imbalanced, which will pose a challenge. (I'll go ahead and reveal that the `dev` set is similarly distributed.)

In [10]:
def label_distribution(split):
    return pd.DataFrame(wordentail_data[split]['train'])[1].value_counts()

In [11]:
label_distribution('edge_disjoint')

synonym     8865
hypernym    6475
hyponym     1044
antonym      629
Name: 1, dtype: int64

### Word disjoint

In [12]:
wordentail_data['word_disjoint'].keys()

dict_keys(['dev', 'train'])

In the `word_disjoint` split, no __words__ are shared between `train` and `dev`:

In [13]:
nli.get_vocab_overlap_size(wordentail_data, 'word_disjoint')

0

Because no words are shared between `train` and `dev`, no edges are either:

In [14]:
nli.get_edge_overlap_size(wordentail_data, 'word_disjoint')

0

The label distribution is similar to that of `edge_disjoint`, though the overall number of examples is a bit smaller:

In [15]:
label_distribution('word_disjoint')

synonym     5610
hypernym    3993
hyponym      627
antonym      386
Name: 1, dtype: int64

There is still an important bias in the data: some words appear much more often than others, and in specific positions. For example, the very general term `part` appears on the right in a large number of cases, many of them `hypernym`.

In [16]:
[[ex, y] for ex, y in wordentail_data['word_disjoint']['train'] 
 if ex[1] == 'part']

[[['frames', 'part'], 'hypernym'],
 [['heaven', 'part'], 'hypernym'],
 [['pan', 'part'], 'synonym'],
 [['middle', 'part'], 'hypernym'],
 [['shared', 'part'], 'synonym'],
 [['shares', 'part'], 'synonym'],
 [['ended', 'part'], 'hypernym'],
 [['twin', 'part'], 'synonym'],
 [['meal', 'part'], 'synonym'],
 [['bit', 'part'], 'hypernym'],
 [['sections', 'part'], 'synonym'],
 [['capacity', 'part'], 'hypernym'],
 [['beginning', 'part'], 'hypernym'],
 [['divorce', 'part'], 'hypernym'],
 [['paradise', 'part'], 'hypernym'],
 [['ends', 'part'], 'hypernym'],
 [['reduced', 'part'], 'hypernym'],
 [['units', 'part'], 'hypernym'],
 [['corner', 'part'], 'hypernym'],
 [['air', 'part'], 'hypernym'],
 [['section', 'part'], 'synonym'],
 [['something', 'part'], 'synonym'],
 [['reduce', 'part'], 'hypernym'],
 [['some', 'part'], 'synonym'],
 [['heavy', 'part'], 'hypernym'],
 [['segment', 'part'], 'hypernym'],
 [['share', 'part'], 'synonym'],
 [['hat', 'part'], 'hypernym'],
 [['maria', 'part'], 'hypernym'],
 [['

These tabulations suggest that a classifier could do well just by learning where words tend to appear:

In [17]:
def count_label_position_instances(split, pos=0):
    examples = wordentail_data[split]['train']    
    return pd.Series([(ex[pos], label) for ex, label in examples]).value_counts()

In [18]:
count_label_position_instances('word_disjoint', pos=0).head()

(forms, hypernym)    9
(test, hypernym)     8
(tests, hypernym)    8
(items, synonym)     8
(gang, synonym)      8
dtype: int64

In [19]:
count_label_position_instances('word_disjoint', pos=1).head()

(be, hypernym)        51
(take, hypernym)      39
(alter, hypernym)     38
(person, hypernym)    33
(modify, hypernym)    32
dtype: int64

### Word disjoint and balanced

To see how much our models are leveraging the uneven distribution of words across the left and right positions, we also have a split in which each word $w$ appears in at most one item $((w, w_{R}), y)$ and at most one item $((w_{L}, w), y)$.

The following tests establish that the dataset has the desired properties:

In [20]:
wordentail_data['word_disjoint_balanced'].keys()

dict_keys(['dev', 'train'])

In [21]:
nli.get_edge_overlap_size(wordentail_data, 'word_disjoint_balanced')

0

In [22]:
nli.get_vocab_overlap_size(wordentail_data, 'word_disjoint_balanced')

0

In [23]:
[[ex, y] for ex, y in wordentail_data['word_disjoint_balanced']['train'] 
 if ex[1] == 'part']

[[['frames', 'part'], 'hypernym'], [['pan', 'part'], 'synonym']]

In [24]:
count_label_position_instances('word_disjoint_balanced', pos=0).head()

(shirts, hypernym)     1
(milk, hypernym)       1
(compiled, synonym)    1
(four, hypernym)       1
(bound, antonym)       1
dtype: int64

In [25]:
count_label_position_instances('word_disjoint_balanced', pos=1).head()

(doing, synonym)          1
(outside, synonym)        1
(snap, hypernym)          1
(talking, hypernym)       1
(convertible, synonym)    1
dtype: int64

## Baseline

Even in deep learning, __feature representation is the most important thing and requires care!__
For our task, feature representation has two parts: representing the individual words and combining those representations into a single network input.

### Representing words: vector_func

Let's consider two baseline word representations methods:

1. Random vectors (as returned by `utils.randvec`).
1. 50-dimensional GloVe representations.

In [26]:
def randvec(w, n=50, lower=-1.0, upper=1.0):
    """Returns a random vector of length `n`. `w` is ignored."""
    return utils.randvec(n=n, lower=lower, upper=upper)

In [29]:
# Any of the files in glove.6B will work here:
glove50_src = os.path.join(glove_home, 'glove.6B.50d.txt')

# Creates a dict mapping strings (words) to GloVe vectors:
GLOVE50 = utils.glove2dict(glove50_src)

def glove50vec(w):    
    """Return `w`'s GloVe representation if available, else return 
    a random vector."""
    return GLOVE50.get(w, randvec(w, n=50))

### Combining words into inputs: vector_combo_func

Here we decide how to combine the two word vectors into a single representation. In more detail, where `u` is a vector representation of the left word and `v` is a vector representation of the right word, we need a function `vector_combo_func` such that `vector_combo_func(u, v)` returns a new input vector `z` of dimension `m`. A simple example is concatenation:

In [30]:
def vec_concatenate(u, v):
    """Concatenate np.array instances `u` and `v` into a new np.array"""
    return np.concatenate((u, v))

`vector_combo_func` could instead be vector average, vector difference, etc. (even combinations of those) – there's lots of space for experimentation here.

### Classifier model

For a baseline model, I chose `TfShallowNeuralClassifier` with a pretty large hidden layer and a correspondingly high number of iterations. 

In [31]:
net = TfShallowNeuralClassifier(hidden_dim=200, max_iter=500)

### Baseline results

The following puts the above pieces together, using `vector_func=glove50vec`, since `vector_func=randvec` seems so hopelessly misguided for `word_disjoint` and `word_disjoint_balanced`!

First, we build the dataset:

In [32]:
X = nli.build_bakeoff_dataset(
    wordentail_data, 
    vector_func=glove50vec,
    vector_combo_func=vec_concatenate)

And then we run the experiment with `nli.bakeoff_experiment`. This trains and tests on all three splits, and additionally trains on `word_disjoint`'s `train` portion and tests on `word_disjoint_balanced`'s `dev` portion, to see what distribution of examples is more effective for this balanced evaluation.

Since the bake-off focus is `word_disjoint`, you might want to run just that evaluation. To to that, use:

In [33]:
nli.bakeoff_experiment(X, net, conditions=['word_disjoint'])

Iteration 500: loss: 9.8029803037643437

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.53      0.44      0.48      1594
    hyponym       0.09      0.00      0.01       275
    synonym       0.59      0.77      0.67      2229

avg / total       0.51      0.57      0.53      4248



/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


This will run the complete evaluation:

In [34]:
nli.bakeoff_experiment(X, net)

/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iteration 1: loss: 15.40992927551269592

edge_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       392
   hypernym       0.58      0.44      0.50      4310
    hyponym       0.49      0.04      0.07       710
    synonym       0.59      0.80      0.68      5930

avg / total       0.56      0.59      0.55     11342



Iteration 500: loss: 9.8474014997482318/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.55      0.41      0.47      1594
    hyponym       0.25      0.01      0.03       275
    synonym       0.58      0.79      0.67      2229

avg / total       0.53      0.57      0.53      4248



Iteration 500: loss: 3.0677949190139773

word_disjoint_balanced
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       115
   hypernym       0.48      0.29      0.36       511
    hyponym       0.75      0.03      0.05       118
    synonym       0.55      0.84      0.67       831

avg / total       0.50      0.54      0.47      1575



Iteration 500: loss: 9.8080876469612126

word_disjoint_balanced, training on word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       115
   hypernym       0.47      0.39      0.43       511
    hyponym       0.14      0.01      0.02       118
    synonym       0.58      0.80      0.67       831

avg / total       0.47      0.55      0.49      1575



## Bake-off submission

__The goal__: achieve the highest average F1 score on __word_disjoint__.

__Submit:__

* Your score on the `word_disjoint` split.
* A description of the method you used: 
   * Your approach to representing words.
   * Your approach to combining them into inputs.
   * The model you used for predictions.
   
__Submission URL__: https://goo.gl/forms/CizXwS3kfPjsThxA3   

__Notes:__

* For the methods, the only requirement is that they differ in some way from the baseline above. They don't have to be completely different, though. For example, you might want to stick with the model but represent examples differently, or the reverse.

* You must train only on the `train` split. No outside training instances can be brought in. You can, though, bring in outside information via your input vectors, as long as this information is not from `dev` or `edge_disjoint`. 

* You can also augment your training data. For example, if `((A, B), synonym)` is a training instance, then so should be `((B, A), synonym)`. Similarly, `((A, B), hyponym)`  and `((B, C), hyponym)` are training cases, then so should be `((A, C), hyponym)`.

* Since the evaluation is for `word_disjoint`, you're not going to get very far with random input vectors! A GloVe featurizer is defined above. Feel free to look around for new word vectors on the Web, or even train your own using our VSM notebooks.

* You're not required to stick to `TfShallowNeuralNetwork`. For instance, you could create deeper feed-forward networks, change how they optimize, etc. As long as you have `fit` and `predict` methods with the same input and output types as our networks, you should be able to use `bakeoff_experiment`. For notes on how to extend the TensorFlow models included in this repository, see [tensorflow_models.ipynb](tensorflow_models.ipynb).

## Augment Data

In [ ]:
with open(wordentail_filename) as f:
    wordentail_data = json.load(f)

In [67]:
train_data = wordentail_data['word_disjoint']['train']
print(len(train_data))
train_data[:5]

10616


[[['life', 'spirit'], 'synonym'],
 [['effective', 'competent'], 'synonym'],
 [['fears', 'panic'], 'hyponym'],
 [['off', 'in'], 'antonym'],
 [['restoration', 'model'], 'hypernym']]

In [68]:
data_to_add = []
for (word1, word2), relation in train_data:
    if relation in ['synonym', 'antonym']:
        data_to_add.append([[word2, word1], relation])
    elif relation == 'hyponym':
        data_to_add.append([[word2, word1], 'hypernym'])
    elif relation == 'hypernym':
        data_to_add.append([[word2, word1], 'hyponym'])
    else:
        raise
        
train_lookup = {}
for (word1, word2), relation in train_data:
    if not relation in train_lookup:
        train_lookup[relation] = {}
    if not word1 in train_lookup[relation]:
        train_lookup[relation][word1] = set()
    if not word2 in train_lookup[relation][word1]:
        train_lookup[relation][word1].add(word2)
        
for data in data_to_add:
    (word1, word2), relation = data
    if word1 not in train_lookup[relation] or word2 not in train_lookup[relation][word1]:
        print(data)
        train_data.append(data)
        
print(len(wordentail_data['word_disjoint']['train']))

[['spirit', 'life'], 'synonym']
[['competent', 'effective'], 'synonym']
[['panic', 'fears'], 'hypernym']
[['in', 'off'], 'antonym']
[['model', 'restoration'], 'hyponym']
[['charged', 'mortgage'], 'synonym']
[['position', 'bed'], 'hyponym']
[['group', 'stack'], 'synonym']
[['wet', 'shower'], 'synonym']
[['modify', 'antique'], 'hyponym']
[['assembly', 'groups'], 'synonym']
[['man', 'sir'], 'hyponym']
[['beginning', 'birth'], 'synonym']
[['adolescent', 'youth'], 'synonym']
[['mild', 'moderate'], 'synonym']
[['tell', 'read'], 'synonym']
[['stated', 'declared'], 'synonym']
[['consider', 'received'], 'hyponym']
[['analyze', 'surveys'], 'hyponym']
[['precise', 'accurate'], 'synonym']
[['remove', 'discharge'], 'hyponym']
[['hero', 'legend'], 'synonym']
[['sex', 'loves'], 'hyponym']
[['sail', 'navigate'], 'synonym']
[['divide', 'formats'], 'hyponym']
[['roller', 'truck'], 'synonym']
[['training', 'discipline'], 'hyponym']
[['include', 'containing'], 'hyponym']
[['remove', 'export'], 'synonym']


[['engineer', 'jones'], 'hyponym']
[['likelihood', 'chances'], 'synonym']
[['sequence', 'opening'], 'hyponym']
[['sacrifice', 'host'], 'synonym']
[['mind', 'forget'], 'antonym']
[['foul', 'dirty'], 'synonym']
[['placed', 'located'], 'synonym']
[['fly', 'break'], 'hyponym']
[['make', 'produce'], 'synonym']
[['household', 'many'], 'synonym']
[['basement', 'stories'], 'synonym']
[['pay', 'found'], 'hyponym']
[['express', 'represent'], 'synonym']
[['consist', 'lies'], 'synonym']
[['fabric', 'felt'], 'hyponym']
[['shape', 'pace'], 'hyponym']
[['angel', 'oracle'], 'synonym']
[['rising', 'climbing'], 'hyponym']
[['wild', 'romantic'], 'synonym']
[['stop', 'checking'], 'hyponym']
[['retain', 'continue'], 'synonym']
[['bear', 'produced'], 'synonym']
[['stick', 'contact'], 'hypernym']
[['judgment', 'voices'], 'synonym']
[['prophet', 'samuel'], 'hyponym']
[['course', 'gang'], 'synonym']
[['earn', 'gross'], 'hyponym']
[['intelligence', 'updated'], 'hyponym']
[['rank', 'state'], 'synonym']
[['statio

[['continually', 'always'], 'synonym']
[['engage', 'promised'], 'synonym']
[['shoot', 'flight'], 'hyponym']
[['introductory', 'preliminary'], 'synonym']
[['stand', 'stands'], 'synonym']
[['who', 'which'], 'synonym']
[['couple', 'joining'], 'synonym']
[['bound', 'hop'], 'hyponym']
[['pressure', 'weight'], 'synonym']
[['cover', 'paint'], 'hyponym']
[['author', 'walker'], 'hyponym']
[['pour', 'stream'], 'synonym']
[['break', 'divorce'], 'hyponym']
[['cause', 'put'], 'synonym']
[['renew', 'recruiting'], 'synonym']
[['controlling', 'governmental'], 'synonym']
[['appear', 'sound'], 'hyponym']
[['birth', 'expect'], 'hyponym']
[['sole', 'soul'], 'synonym']
[['shelter', 'shield'], 'synonym']
[['sink', 'fall'], 'synonym']
[['legal', 'moral'], 'antonym']
[['lady', 'hamilton'], 'hyponym']
[['remove', 'bone'], 'hyponym']
[['communicate', 'issue'], 'hyponym']
[['party', 'role'], 'synonym']
[['unusual', 'odd'], 'synonym']
[['wanting', 'missing'], 'synonym']
[['ray', 'beam'], 'synonym']
[['shelter', '

[['set', 'roots'], 'hyponym']
[['direct', 'navigate'], 'hyponym']
[['cow', 'buffalo'], 'hyponym']
[['apart', 'broken'], 'synonym']
[['extension', 'reached'], 'synonym']
[['accurately', 'approximately'], 'antonym']
[['repeat', 'recorded'], 'synonym']
[['clear', 'discharge'], 'synonym']
[['contest', 'cooperation'], 'antonym']
[['undertake', 'guarantees'], 'synonym']
[['travel', 'hop'], 'hyponym']
[['conclude', 'include'], 'synonym']
[['set', 'provides'], 'hyponym']
[['labor', 'babies'], 'hyponym']
[['modify', 'fat'], 'hyponym']
[['part', 'reduced'], 'hyponym']
[['exclusion', 'exception'], 'synonym']
[['heart', 'regards'], 'hyponym']
[['vagina', 'pussy'], 'synonym']
[['kind', 'milf'], 'synonym']
[['leading', 'main'], 'synonym']
[['plot', 'patches'], 'synonym']
[['layer', 'walls'], 'hyponym']
[['sport', 'entertainment'], 'synonym']
[['discharge', 'fill'], 'synonym']
[['cord', 'cables'], 'synonym']
[['agree', 'subscribe'], 'hyponym']
[['bound', 'preparing'], 'synonym']
[['chart', 'map'], 's

[['yarn', 'picks'], 'hyponym']
[['precisely', 'truly'], 'synonym']
[['bare', 'hairy'], 'antonym']
[['many', 'some'], 'antonym']
[['load', 'pills'], 'hyponym']
[['active', 'living'], 'synonym']
[['state', 'status'], 'synonym']
[['rely', 'found'], 'synonym']
[['apparent', 'actual'], 'antonym']
[['competent', 'fit'], 'synonym']
[['nil', 'zero'], 'synonym']
[['carry', 'features'], 'hypernym']
[['correction', 'spanking'], 'hyponym']
[['venture', 'risk'], 'synonym']
[['office', 'trade'], 'synonym']
[['permit', 'allowed'], 'hyponym']
[['concern', 'article'], 'synonym']
[['bulk', 'quantity'], 'synonym']
[['exact', 'requires'], 'synonym']
[['paper', 'carbon'], 'hyponym']
[['take', 'shell'], 'hyponym']
[['obligation', 'job'], 'hyponym']
[['angry', 'pleased'], 'antonym']
[['introduce', 'present'], 'hyponym']
[['amend', 'advance'], 'hyponym']
[['both', 'either'], 'synonym']
[['execute', 'act'], 'synonym']
[['sally', 'started'], 'synonym']
[['painting', 'landscape'], 'hyponym']
[['intent', 'studied

[['divine', 'trinity'], 'hyponym']
[['analyze', 'reviews'], 'hyponym']
[['advance', 'grow'], 'synonym']
[['projection', 'rising'], 'synonym']
[['accept', 'receives'], 'hyponym']
[['get', 'take'], 'hyponym']
[['hollow', 'pit'], 'synonym']
[['function', 'operation'], 'synonym']
[['chief', 'primary'], 'synonym']
[['waste', 'lose'], 'synonym']
[['portion', 'pan'], 'synonym']
[['someone', 'sport'], 'hyponym']
[['adopt', 'affiliate'], 'synonym']
[['little', 'bit'], 'synonym']
[['detail', 'point'], 'synonym']
[['station', 'standing'], 'synonym']
[['guide', 'direct'], 'synonym']
[['count', 'total'], 'hyponym']
[['be', 'take'], 'hyponym']
[['compound', 'accounts'], 'hypernym']
[['adjust', 'balance'], 'synonym']
[['gauge', 'sizes'], 'synonym']
[['way', 'course'], 'synonym']
[['dependent', 'independent'], 'antonym']
[['accomplished', 'seen'], 'synonym']
[['win', 'get'], 'synonym']
[['upset', 'defeat'], 'hypernym']
[['exposed', 'nude'], 'synonym']
[['place', 'date'], 'synonym']
[['variety', 'style

[['just', 'square'], 'synonym']
[['divide', 'partition'], 'synonym']
[['mixture', 'temperature'], 'synonym']
[['direct', 'point'], 'hyponym']
[['production', 'job'], 'hyponym']
[['import', 'effects'], 'hyponym']
[['defend', 'protected'], 'synonym']
[['place', 'region'], 'synonym']
[['point', 'dates'], 'hyponym']
[['projection', 'holes'], 'antonym']
[['way', 'parking'], 'hyponym']
[['delight', 'satisfy'], 'hypernym']
[['distance', 'maintaining'], 'hypernym']
[['grounds', 'tracks'], 'hyponym']
[['system', 'government'], 'hyponym']
[['ticket', 'labels'], 'synonym']
[['plot', 'action'], 'hyponym']
[['chosen', 'elite'], 'synonym']
[['place', 'behalf'], 'hyponym']
[['top', 'cap'], 'synonym']
[['advance', 'increases'], 'synonym']
[['discover', 'revealed'], 'synonym']
[['cover', 'sheet'], 'hyponym']
[['reward', 'pays'], 'synonym']
[['control', 'steering'], 'hyponym']
[['pocket', 'bag'], 'synonym']
[['quantity', 'proof'], 'hyponym']
[['affect', 'visits'], 'synonym']
[['manner', 'course'], 'syno

[['bottom', 'top'], 'antonym']
[['hit', 'glance'], 'hyponym']
[['infant', 'babies'], 'synonym']
[['worker', 'rat'], 'hyponym']
[['debt', 'scores'], 'hyponym']
[['result', 'outcome'], 'synonym']
[['work', 'layer'], 'hyponym']
[['shelter', 'screens'], 'synonym']
[['charge', 'scores'], 'synonym']
[['apparel', 'shirt'], 'hyponym']
[['sphere', 'realm'], 'synonym']
[['cost', 'rates'], 'synonym']
[['hint', 'suggest'], 'synonym']
[['motor', 'cars'], 'synonym']
[['somebody', 'cancer'], 'hyponym']
[['vast', 'broad'], 'synonym']
[['guardian', 'monitors'], 'hyponym']
[['total', 'average'], 'hyponym']
[['issue', 'outcome'], 'synonym']
[['exactly', 'truly'], 'synonym']
[['condition', 'situation'], 'synonym']
[['existence', 'essential'], 'synonym']
[['explicit', 'express'], 'synonym']
[['try', 'trials'], 'hyponym']
[['employ', 'address'], 'hyponym']
[['brief', 'compact'], 'synonym']
[['control', 'dial'], 'hyponym']
[['colour', 'yellow'], 'hyponym']
[['gain', 'getting'], 'synonym']
[['cheat', 'tricks'

[['timber', 'color'], 'hyponym']
[['formation', 'creation'], 'synonym']
[['divide', 'segments'], 'synonym']
[['walk', 'exhibit'], 'hyponym']
[['agreement', 'confirmation'], 'hyponym']
[['expression', 'term'], 'synonym']
[['football', 'rugby'], 'hyponym']
[['constitutional', 'legal'], 'synonym']
[['post', 'stocks'], 'synonym']
[['route', 'circuit'], 'hyponym']
[['still', 'console'], 'hypernym']
[['deviation', 'variation'], 'synonym']
[['fire', 'low'], 'synonym']
[['day', 'afternoon'], 'hyponym']
[['proportion', 'rates'], 'hyponym']
[['parade', 'display'], 'synonym']
[['flush', 'heat'], 'synonym']
[['hero', 'champion'], 'synonym']
[['propose', 'designs'], 'synonym']
[['budget', 'computing'], 'hypernym']
[['person', 'virgin'], 'hyponym']
[['knit', 'tie'], 'synonym']
[['strength', 'main'], 'synonym']
[['look', 'eye'], 'hyponym']
[['pain', 'please'], 'antonym']
[['one', 'some'], 'synonym']
[['lean', 'trend'], 'synonym']
[['climb', 'ride'], 'hyponym']
[['close', 'far'], 'antonym']
[['sign', 

In [70]:
X = nli.build_bakeoff_dataset(
    wordentail_data, 
    vector_func=glove50vec,
    vector_combo_func=vec_concatenate)

In [71]:
nli.bakeoff_experiment(X, net, conditions=['word_disjoint'])

/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iteration 500: loss: 20.003816664218903

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.61      0.24      0.35      1594
    hyponym       0.15      0.17      0.16       275
    synonym       0.56      0.83      0.67      2229

avg / total       0.53      0.54      0.49      4248



/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Difference of Vectors

In [72]:
def vec_diff(u, v):
    return u - v

In [73]:
with open(wordentail_filename) as f:
    wordentail_data = json.load(f)

In [74]:
X = nli.build_bakeoff_dataset(
    wordentail_data, 
    vector_func=glove50vec,
    vector_combo_func=vec_diff)

In [75]:
nli.bakeoff_experiment(X, net, conditions=['word_disjoint'])

/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iteration 500: loss: 10.296298563480377

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.55      0.30      0.39      1594
    hyponym       0.00      0.00      0.00       275
    synonym       0.56      0.84      0.67      2229

avg / total       0.50      0.56      0.50      4248



/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Bigger Glove Vectors

In [76]:
# Any of the files in glove.6B will work here:
glove100_src = os.path.join(glove_home, 'glove.6B.100d.txt')

# Creates a dict mapping strings (words) to GloVe vectors:
GLOVE100 = utils.glove2dict(glove100_src)

def glove100vec(w):    
    """Return `w`'s GloVe representation if available, else return 
    a random vector."""
    return GLOVE100.get(w, randvec(w, n=100))

In [77]:
X = nli.build_bakeoff_dataset(
    wordentail_data, 
    vector_func=glove100vec,
    vector_combo_func=vec_concatenate)

In [78]:
nli.bakeoff_experiment(X, net, conditions=['word_disjoint'])

/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iteration 500: loss: 9.066815853118896

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.56      0.49      0.52      1594
    hyponym       0.39      0.06      0.10       275
    synonym       0.60      0.76      0.67      2229

avg / total       0.55      0.58      0.55      4248



In [82]:
# Any of the files in glove.6B will work here:
glove200_src = os.path.join(glove_home, 'glove.6B.200d.txt')

# Creates a dict mapping strings (words) to GloVe vectors:
GLOVE200 = utils.glove2dict(glove200_src)

def glove200vec(w):    
    """Return `w`'s GloVe representation if available, else return 
    a random vector."""
    return GLOVE200.get(w, randvec(w, n=200))

In [83]:
X = nli.build_bakeoff_dataset(
    wordentail_data, 
    vector_func=glove200vec,
    vector_combo_func=vec_concatenate)

In [84]:
nli.bakeoff_experiment(X, net, conditions=['word_disjoint'])

/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iteration 500: loss: 8.938477993011475

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.53      0.44      0.48      1594
    hyponym       0.24      0.03      0.05       275
    synonym       0.58      0.75      0.66      2229

avg / total       0.52      0.56      0.53      4248



## MOAR LAYERZZZ

In [85]:
net = TfShallowNeuralClassifier(hidden_dim=100, max_iter=500)

In [86]:
X = nli.build_bakeoff_dataset(
    wordentail_data, 
    vector_func=glove100vec,
    vector_combo_func=vec_concatenate)

In [87]:
nli.bakeoff_experiment(X, net, conditions=['word_disjoint'])

/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iteration 500: loss: 9.1561388373374942

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.55      0.50      0.53      1594
    hyponym       0.38      0.05      0.10       275
    synonym       0.60      0.74      0.66      2229

avg / total       0.55      0.58      0.55      4248



In [88]:
net = TfShallowNeuralClassifier(hidden_dim=400, max_iter=500)

In [89]:
nli.bakeoff_experiment(X, net, conditions=['word_disjoint'])

/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iteration 500: loss: 9.150443315505981

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.56      0.47      0.51      1594
    hyponym       0.37      0.08      0.14       275
    synonym       0.60      0.77      0.67      2229

avg / total       0.55      0.58      0.55      4248



In [90]:
net = TfShallowNeuralClassifier(hidden_dim=50, max_iter=500)

In [91]:
nli.bakeoff_experiment(X, net, conditions=['word_disjoint'])

/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iteration 500: loss: 9.3420071601867684

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.57      0.49      0.53      1594
    hyponym       0.39      0.05      0.08       275
    synonym       0.60      0.76      0.67      2229

avg / total       0.55      0.59      0.55      4248



## Non-Deep Learning Models?

In [92]:
from sklearn.ensemble import RandomForestClassifier

In [93]:
clf = RandomForestClassifier()

In [94]:
nli.bakeoff_experiment(X, clf, conditions=['word_disjoint'])

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.43      0.38      0.41      1594
    hyponym       0.06      0.00      0.01       275
    synonym       0.55      0.69      0.61      2229

avg / total       0.45      0.50      0.47      4248



In [95]:
clf = RandomForestClassifier(n_estimators=50)

In [96]:
nli.bakeoff_experiment(X, clf, conditions=['word_disjoint'])

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.54      0.27      0.36      1594
    hyponym       0.00      0.00      0.00       275
    synonym       0.56      0.86      0.68      2229

avg / total       0.49      0.55      0.49      4248



/Users/mattwilber/.pyenv/versions/3.5.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
